# Crawller - Psicologos 
## Crawler em largura
Este notebook dispõe de um crawller para páginas contendo instâncias de psicologos, psiquiatras, clínicas e outros profissionais neste domínio.

In [2]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
import time
import random

## Sementes

Partiremos de "sementes", que são sites dos quais já analisamos manualmente anteriormente e é sabido que levam a instâncias de profissionais do nosso domínio.

In [3]:
df = pd.read_csv('sementes.csv')
df

,semente,dominio,robotstxt,localpath,htmlname
0,https://www.srlupa.com.br,srlupa.com.br,False,pages/sementes/srlupa/,srlupa
1,https://serviceira.com,serviceira.com,False,pages/sementes/serviceira/,serviceira
2,https://conectu.com.br,conectu.com.br,False,pages/sementes/conectu/,conectu
3,https://www.achouservicos.com.br,achouservicos.com.br,False,pages/sementes/achouservicos/,achouservicos
4,https://www.telelistas.net/br/clinicas+de+psic...,telelistas.net,True,pages/sementes/telelistas/clinicasdepsicologia/,clinicasdepsicologia
5,https://www.telelistas.net/br/clinicas+psiquia...,telelistas.net,True,pages/sementes/telelistas/clinicaspsiquiatricas/,clinicaspsiquiatricas
6,https://www.telelistas.net/br/psicanalistas,telelistas.net,True,pages/sementes/telelistas/psicanalistas/,psicanalistas
7,https://www.telelistas.net/br/psicologos,telelistas.net,True,pages/sementes/telelistas/psicologos/,psicologos
8,https://www.telelistas.net/br/psicopedagogos,telelistas.net,True,pages/sementes/telelistas/psicopedagogos/,psicopedagogos
9,https://freelancer.com.br,freelancer.com.br,True,pages/sementes/freelancer/,freelancer


## Robots.txt

Nossa primeira preocupação é respeitar o robots.txt disponível no site que vamos crawllear.
Há bibliotecas para tal funcionalidade. Para os domínios das sementes já havia, previamente, baixado teus robots.txt (quando existia).

O metodo getDisallows(path) abaixo recebe o path onde está a semente e o robots.txt e, iterando sobre ele, retorna a lista de
restrições do robots.txt.

In [4]:
def getDisallows(path):
    ret_list = []
    file = open(path+"robots.txt", "r")
    lines = file.readlines()
    read = False
    for line in lines:
        if(line.endswith("\n")):
            line = line[:-1]
        if(line.lower().startswith("user-agent:") and line.endswith("*")):
            read = True
        elif(line.lower().startswith("user-agent:") and not(line.endswith("*"))):
            read = False
        elif(line.lower().startswith("disallow:") and read):
            aux = line[10:]
            
            if(aux.endswith("*")):
                aux = aux[:-1]
            if(aux.endswith("?")):
                aux = aux[:-1]
            if(aux.endswith("?")):
                aux = aux[:-1]
            if(aux.endswith("*")):
                aux = aux[:-1]
                
            ret_list.append(aux)
    return ret_list

## Crawleando em largura

Neste primeiro momento, rodo o Crawller partindo de cada semente, iterativamente. É uma busca em largura, onde - por
motivos de eficiencia - só entro numa página uma vez, só acesso links .html e checo se estou dentro dos domínios das sementes (para evitar sair do domínio para um site como, por exemplo, google.com).
Também respeitaremos os sites dando um time entre uma requisição e a próxima.

In [ ]:
time_to_sleep = 1
limiar = 1000
dict_link_accessed = {}

def putLinkToAccessList(domain, link):
    link = link.get('href')
    if link is None:
        return link_to_access_list
    
    if domain in link:
        if (link not in link_to_access_list):
            link_to_access_list.append(link)
            return link_to_access_list
    elif link.startswith("/"):
        if (domain+link not in link_to_access_list):
            link_to_access_list.append(domain+link)
            return link_to_access_list
     
    
for i in range(len(df)):
    domain_block_list = []
    link_accessed_list = []
    link_to_access_list = []
    #iterando na semente
    domain = df['dominio'][i]
    print("Crawleando a semente:\t"+domain)
    link_accessed_list.append(df['semente'][i])    
    path = df['localpath'][i]
    
    if(df['robotstxt'][i]):
        domain_block_list = getDisallows(path)
        print("Esta semente contém um Robots.txt com restrições em:\t"+str(domain_block_list))
        
        
    url = path+df['htmlname'][i]+".html"
    parsed_page = BeautifulSoup(open(url,encoding="latin1"),"html.parser")
    print("Iniciando Crawler nesta semente em:\t"+str(url))
    total_links = parsed_page.find_all('a')
    print("Total de links na página:\t"+str(len(total_links)))

    lenght_front_count_aux = len(link_to_access_list)
    
    
    for link in total_links:
        putLinkToAccessList(domain,link)                           
                            
    lenght_front_count_aux = len(link_to_access_list) - lenght_front_count_aux
    print("Total de links adicionados a fronteira:\t"+str(lenght_front_count_aux))
    
    time_to_sleep_aux = random.randint(1,time_to_sleep)
    print("Aguardando "+str(time_to_sleep_aux)+" segundos para a próxima requisição")
    #time.sleep(time_to_sleep_aux)
    
    for link in link_to_access_list:
        if(not link.startswith("http://") and not link.startswith("https://")):
            link = "http://"+link
        print("Tentando acessar "+link+"   ...")
        block = False
        for blockitem in domain_block_list:
            if(blockitem in link):
                block = True
                print("URL bloqueada pelo robots.txt")
                break
        if(not block and link not in link_accessed_list):
            try:
                ret=rq.get(link)
                if 'text/html' in ret.headers['Content-Type']:
                    print("Página acessada com sucesso")
                    link_accessed_list.append(link)
                    ret.text

                    if len(link_accessed_list)==limiar:
                        print("LIMIAR ATINGIDO! Foram acessadas "+str(limiar)+" links nesta semente\n\n")
                        dict_link_accessed[domain] = link_accessed_list
                        break

                    parsed_page = BeautifulSoup(ret.text,"html.parser")
                    print("Capturando links para a fronteira")
                    total_links = parsed_page.find_all('a')
                    print("Total de links na página:\t"+str(len(total_links)))

                    lenght_front_count_aux = len(link_to_access_list)

                    for linkItem in total_links:
                        putLinkToAccessList(domain, linkItem)

                    lenght_front_count_aux = len(link_to_access_list) - lenght_front_count_aux

                    print("Total de links adicionados a fronteira:\t"+str(lenght_front_count_aux))
                    print("Tamanho atual da fronteira:\t"+str(len(link_to_access_list)))
                    print("Crawler concluiu sua passada por "+str(link)+"   ...")
                else:
                    print("O conteúdo do link não é html. O Crawler continuará do próximo na fronteira!")
            except:
                print("Erro ao acessar o link "+str(link)+"\nO Crawler continuará do próximo na fronteira!")
        
        time_to_sleep_aux = random.randint(1,time_to_sleep)
        print("Aguardando "+str(time_to_sleep_aux)+" segundos para a próxima requisição")
        #time.sleep(time_to_sleep_aux)
        
    dict_link_accessed[domain] = link_accessed_list                
    print("Crawler concluiu atividade na semente "+domain+"   ...\n")
    print("\n---------------------\n----\n---------------------\n\n")

print("Crawler finalizou sua atividade")

Crawleando a semente:	srlupa.com.br
Iniciando Crawler nesta semente em:	pages/sementes/srlupa/srlupa.html
Total de links na página:	72
Total de links adicionados a fronteira:	35
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	72
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	35
Crawler concluiu sua passada por http://srlupa.com.br/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/accounts/login/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	46
Total de links adicionados a fronteira:	2
Tamanho atual da fronteira:	37
Crawler concluiu sua passada por http://srlupa.com.br/accounts/login/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/ofereca-seus-servicos/   ...
Página acessada com sucesso
Capturando li

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	61
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	431
Crawler concluiu sua passada por http://srlupa.com.br/sobre/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/central-de-ajuda/perguntas-frequentes/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	81
Total de links adicionados a fronteira:	1
Tamanho atual da fronteira:	432
Crawler concluiu sua passada por http://srlupa.com.br/central-de-ajuda/perguntas-frequentes/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/cadastro-sr-lupa/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	48
Total de links adicionados a fronteira:	2
Tamanho atual da fronteira:	434
Crawler concluiu sua passada por http://srlupa.com.br/cadastro-sr-lupa/   ...
Aguardando 1 segundo

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	308
Total de links adicionados a fronteira:	33
Tamanho atual da fronteira:	1255
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/assistencia-tecnica/ar-condicionado/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/assistencia-tecnica/aspirador-de-po/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	61
Total de links adicionados a fronteira:	1
Tamanho atual da fronteira:	1256
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/assistencia-tecnica/aspirador-de-po/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/assistencia-tecnica/camera/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	66
Total de links adicionados a fronteira:	2
Tamanho atual da fronteira:	1258
C

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	130
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	1333
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/assistencia-tecnica/micro-ondas/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/assistencia-tecnica/notebook-laptop/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	471
Total de links adicionados a fronteira:	2
Tamanho atual da fronteira:	1335
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/assistencia-tecnica/notebook-laptop/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/assistencia-tecnica/seca-roupa/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	96
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	1335
C

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	58
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	1351
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/aulas/italiana-culinaria/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/aulas/japonesa-culinaria/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	58
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	1351
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/aulas/japonesa-culinaria/   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://srlupa.com.br/profissionais/aulas/mexicana-culinaria/   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	58
Total de links adicionados a fronteira:	1
Tamanho atual da fronteira:	1352
Crawler concluiu sua passada por http:

...

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	303
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27278
Crawler concluiu sua passada por http://telelistas.net/to/palmas   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://telelistas.net/templates/busca_Localidade.aspx   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	241
Total de links adicionados a fronteira:	122
Tamanho atual da fronteira:	27400
Crawler concluiu sua passada por http://telelistas.net/templates/busca_Localidade.aspx   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://telelistas.net/templates/busca_palavrachave.aspx   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	212
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27400
Crawler concluiu sua passada por http://telelistas.net/templates/busca_palavra

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	97
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27406
Crawler concluiu sua passada por http://www.telelistas.net/templates/parceiros.aspx   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar http://www.telelistas.net/templates/form_anuncie_resultado.aspx   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	64
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27406
Crawler concluiu sua passada por http://www.telelistas.net/templates/form_anuncie_resultado.aspx   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/pa/belem/clinicas+de+psicologia   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	297
Total de links adicionados a fronteira:	3
Tamanho atual da fronteira:	27409
Crawler concluiu sua passada por https:/

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	8
Total de links adicionados a fronteira:	6
Tamanho atual da fronteira:	27890
Crawler concluiu sua passada por https://www.telelistas.net/templates/Redireciona.aspx?origem=Facebook&url=https%3A%2F%2Fwww.telelistas.net%2Flocais%2Fpa%2Fbelem%2Fclinicas+de+psicologia%2Fbu-5701998%2Fconsultorio+sandra+menezes+crp+1002564+em+marco&local=controle   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/Redireciona.aspx?origem=Twitter&url=https://www.telelistas.net/locais/pa/belem/clinicas+de+psicologia/bu-5701998/consultorio+sandra+menezes+crp+1002564+em+marco&local=controle&nome=Consultório Sandra Menezes - Crp 10/02564&cidade=PA   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	72
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27890
Crawler concluiu sua passada por https://www.telelistas

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	74
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27913
Crawler concluiu sua passada por https://www.telelistas.net/locais/pa/belem/clinicas+de+psicologia/bu-19803919/cristina+heringer+avaliacao+neuropsicologica+crp+1002928+em+pedreira#contato_email   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/logredirect.aspx?idcliente=0&tpmat=STT&ct=0&mt=0&uf=pa&id=19803919&cidade=91000&bairro=0&titulo=99118&logvc=18&bt=11&acao=whatsapp   ...
Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	188
Total de links adicionados a fronteira:	0
Tamanho atual da fronteira:	27913
Crawler concluiu sua passada por https://www.telelistas.net/templates/logredirect.aspx?idcliente=0&tpmat=STT&ct=0&mt=0&uf=pa&id=19803919&cidade=91000&bairro=0&titulo=99118&logvc=18&bt=11&acao=whatsapp   ...
Aguardando 1 segund

Página acessada com sucesso
Capturando links para a fronteira
Total de links na página:	76
Total de links adicionados a fronteira:	1
Tamanho atual da fronteira:	27930
Crawler concluiu sua passada por https://www.telelistas.net/locais/pa/belem/clinicas+de+psicologia/bu-19603882/alcinda+figueredo+em+umarizal   ...
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/form_login.aspx?ReturnUrl=https://www.telelistas.net/locais/pa/belem/clinicas+de+psicologia/bu-19603882/alcinda+figueredo+em+umarizal#avaliar   ...
URL bloqueada pelo robots.txt
Aguardando 1 segundos para a próxima requisição
Tentando acessar https://www.telelistas.net/templates/logredirect.aspx?idcliente=0&tpmat=AG&ct=0&mt=0&uf=pa&id=19603882&cidade=91000&bairro=0&titulo=99118&logvc=0&bt=3&link=https://www.telelistas.net/locais/pa/belem/clinicas+de+psicologia/bu-19603882/alcinda+figueredo+em+umarizal?initmap   ...
Página acessada com sucesso
Capturando links para a fronteira
T

In [10]:

dict_link_accessed


{'srlupa.com.br': ['https://www.srlupa.com.br',
  'http://srlupa.com.br/',
  'http://srlupa.com.br/accounts/login/',
  'http://srlupa.com.br/ofereca-seus-servicos/',
  'http://srlupa.com.br/categorias/',
  'http://srlupa.com.br/empresa/politica-de-privacidade/',
  'http://srlupa.com.br/empresa/termos-e-condicoes-de-uso/',
  'http://srlupa.com.br/central-de-ajuda/',
  'http://srlupa.com.br/categorias/#category-9',
  'http://srlupa.com.br/categorias/#category-10',
  'http://srlupa.com.br/categorias/#category-12',
  'http://srlupa.com.br/categorias/#category-6',
  'http://srlupa.com.br/categorias/#category-2',
  'http://srlupa.com.br/categorias/#category-3',
  'http://srlupa.com.br/categorias/#category-11',
  'http://srlupa.com.br/categorias/#category-1',
  'http://srlupa.com.br/categorias/#category-5',
  'http://srlupa.com.br/categorias/#category-7',
  'http://srlupa.com.br/categorias/#category-8',
  'http://srlupa.com.br/categorias/#category-4',
  'https://blog.srlupa.com.br/tela-iphone

In [ ]:
len(link_accessed_list)